 ## Machine Learning

## Imports

In [1]:

from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder, VectorIndexer
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator,ClusteringEvaluator
from pyspark.ml.classification import LogisticRegression, DecisionTreeClassifier, RandomForestClassifier, LinearSVC,GBTClassifier
from tabulate import tabulate

## Functions

In [2]:
def evaluate_model(model, train_data, validation_data = None, test_data = None, model_name = "Model", label_column = "MIS_Status"):
    # Evaluate training data
    train_predictions = model.transform(train_data)
    train_metrics = calculate_metrics(train_predictions, label_column)

     # Prepare data for tabulate
    table = [
        ['Model', model_name],
        ['Metric', 'Training'],
        ['Accuracy', train_metrics['accuracy']],
        ['Weighted Precision', train_metrics['weighted_precision']],
        ['Weighted Recall', train_metrics['weighted_recall']],
        ['F1 Score', train_metrics['f1']]
    ]
    # Evaluate validation data
    if validation_data is not None:
        validation_predictions = model.transform(validation_data)
        validation_metrics = calculate_metrics(validation_predictions, label_column)
        table[0] += ['']
        table[1] += ['Validation']
        table[2] += [validation_metrics['accuracy']]
        table[3] += [validation_metrics['weighted_precision']]
        table[4] += [validation_metrics['weighted_recall']]
        table[5] += [validation_metrics['f1']]

    # Evaluate test data
    if test_data is not None:
        test_predictions = model.transform(test_data)
        test_metrics = calculate_metrics(test_predictions, label_column)
        table[0] += ['']
        table[1] += ['Test']
        table[2] += [test_metrics['accuracy']]
        table[3] += [test_metrics['weighted_precision']]
        table[4] += [test_metrics['weighted_recall']]
        table[5] += [test_metrics['f1']]


    # Display results using tabulate
    print(tabulate(table, headers="firstrow", tablefmt='grid'))

def calculate_metrics(predictions, label_column):
    evaluator_multi = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol=label_column, metricName='accuracy')
    evaluator_weighted_precision = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol=label_column, metricName='weightedPrecision')
    evaluator_weighted_recall = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol=label_column, metricName='weightedRecall')
    evaluator_f1 = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol=label_column, metricName='f1')

    accuracy = evaluator_multi.evaluate(predictions)
    weighted_precision = evaluator_weighted_precision.evaluate(predictions)
    weighted_recall = evaluator_weighted_recall.evaluate(predictions)
    f1 = evaluator_f1.evaluate(predictions)

    return {
        'accuracy': accuracy,
        'weighted_precision': weighted_precision,
        'weighted_recall': weighted_recall,
        'f1': f1
    }

In [3]:

spark=SparkSession.builder\
    .master("local[*]")\
    .appName("LoanApproval")\
    .getOrCreate()

sc=spark.sparkContext

 ## Read Data

In [4]:

# data_path="/content/drive/MyDrive/Colab Notebooks/BD_project/50000.csv"
data_path="../sample_data/1000.csv"


In [5]:

loan_df =  spark.read.csv(data_path, header=True, inferSchema=True, multiLine=True, quote='"', escape='"')
# ======================================================
# dropping these columns give better accuracy (by trial)
# ======================================================
loan_df = loan_df.drop('Name')
loan_df = loan_df.drop('Zip')
loan_df = loan_df.drop('City')
# ======================================================
# ======================================================
# ======================================================

In [6]:
loan_df.printSchema()
loan_df.show(5)

root
 |-- State: string (nullable = true)
 |-- Bank: string (nullable = true)
 |-- BankState: string (nullable = true)
 |-- Term: integer (nullable = true)
 |-- NoEmp: integer (nullable = true)
 |-- NewExist: integer (nullable = true)
 |-- CreateJob: integer (nullable = true)
 |-- RetainedJob: integer (nullable = true)
 |-- UrbanRural: integer (nullable = true)
 |-- RevLineCr: integer (nullable = true)
 |-- LowDoc: integer (nullable = true)
 |-- Sector: integer (nullable = true)
 |-- ApprovalMonth: string (nullable = true)
 |-- IsFranchise: integer (nullable = true)
 |-- clean_DisbursementGross: double (nullable = true)
 |-- MIS_Status: integer (nullable = true)
 |-- clean_ChgOffPrinGr: double (nullable = true)
 |-- clean_GrAppv: double (nullable = true)
 |-- clean_SBA_Appv: double (nullable = true)

+-----+--------------------+---------+----+-----+--------+---------+-----------+----------+---------+------+------+-------------+-----------+-----------------------+----------+------------

In [7]:

print("Transforming categorial features...")
# List of categorical columns to be one-hot encoded
# categorical_columns = ["Name", "City", "State", "Zip", "Bank", "BankState", "UrbanRural", "RevLineCr", "LowDoc", "Sector", "ApprovalMonth"]
categorical_columns = ["State", "Bank", "BankState", "UrbanRural", "RevLineCr", "LowDoc", "Sector", "ApprovalMonth"]

# Define an empty list to store the pipeline stages
stages = []

# Iterate over each categorical column
for column in categorical_columns:
    # Define StringIndexer for the current column
    indexer = StringIndexer(inputCol=column, outputCol=column + "Index")

    # Define OneHotEncoder for the indexed column
    encoder = OneHotEncoder(inputCol=column + "Index", outputCol=column + "Vec")

    # Add StringIndexer and OneHotEncoder to the list of stages
    stages += [indexer, encoder]

label_column = "MIS_Status"

# Create VectorAssembler for combining all features
# List of input columns (excluding the label column and categorical columns)
input_columns = [col for col in loan_df.columns if col != label_column and col not in categorical_columns]
input_columns += [column + "Vec" for column in categorical_columns]
assembler = VectorAssembler(inputCols=input_columns , outputCol="features")

# Combine all stages into a Pipeline
pipeline = Pipeline(stages=stages + [assembler])

# Fit the pipeline to your data
pipeline_model = pipeline.fit(loan_df)

# Transform your data using the pipeline
transformed_data = pipeline_model.transform(loan_df)
transformed_data.show(5)
print("Splitting data into training, validation and test...")
# Split the transformed data into training and test sets (60% training, 20% validation, 20% test)
(trainingData, validationData, testData) = transformed_data.randomSplit([0.6, 0.2, 0.2], seed=123)

Transforming categorial features...
+-----+--------------------+---------+----+-----+--------+---------+-----------+----------+---------+------+------+-------------+-----------+-----------------------+----------+------------------+------------+--------------+----------+-------------+---------+-----------------+--------------+--------------+---------------+-------------+--------------+-------------+-----------+-------------+-----------+--------------+------------------+----------------+--------------------+
|State|                Bank|BankState|Term|NoEmp|NewExist|CreateJob|RetainedJob|UrbanRural|RevLineCr|LowDoc|Sector|ApprovalMonth|IsFranchise|clean_DisbursementGross|MIS_Status|clean_ChgOffPrinGr|clean_GrAppv|clean_SBA_Appv|StateIndex|     StateVec|BankIndex|          BankVec|BankStateIndex|  BankStateVec|UrbanRuralIndex|UrbanRuralVec|RevLineCrIndex| RevLineCrVec|LowDocIndex|    LowDocVec|SectorIndex|     SectorVec|ApprovalMonthIndex|ApprovalMonthVec|            features|
+-----+-----

## Logistic Regression

In [8]:
# Create a Logistic Regression model
lr = LogisticRegression(maxIter=10, elasticNetParam=0.8, labelCol=label_column, featuresCol="features")
# Train the model
lrModel = lr.fit(trainingData)

Training logistic regression model...


In [9]:
evaluate_model(lrModel, trainingData, validation_data=validationData, model_name='Logistic Regression', label_column=label_column)

+--------------------+-----------------------+--------------------+
| Model              | Logistic Regression   |                    |
+====================+=======================+====================+
| Metric             | Training              | Validation         |
+--------------------+-----------------------+--------------------+
| Accuracy           | 0.9983164983164983    | 0.9141414141414141 |
+--------------------+-----------------------+--------------------+
| Weighted Precision | 0.9983197483197483    | 0.9094247068385    |
+--------------------+-----------------------+--------------------+
| Weighted Recall    | 0.9983164983164983    | 0.9141414141414141 |
+--------------------+-----------------------+--------------------+
| F1 Score           | 0.9983118099628471    | 0.9107172722883159 |
+--------------------+-----------------------+--------------------+


## Random Forest

In [10]:

 # Create Random Forest model
rf = RandomForestClassifier(featuresCol='features', labelCol=label_column)

# Fit model to training data
rf_model = rf.fit(trainingData)

In [11]:
evaluate_model(rf_model, trainingData, validation_data=validationData, model_name='Random Forest', label_column=label_column)

+--------------------+--------------------+--------------------+
| Model              | Random Forest      |                    |
+====================+====================+====================+
| Metric             | Training           | Validation         |
+--------------------+--------------------+--------------------+
| Accuracy           | 0.9074074074074074 | 0.8686868686868687 |
+--------------------+--------------------+--------------------+
| Weighted Precision | 0.9121426250812217 | 0.8861952861952862 |
+--------------------+--------------------+--------------------+
| Weighted Recall    | 0.9074074074074074 | 0.8686868686868686 |
+--------------------+--------------------+--------------------+
| F1 Score           | 0.8853706284458559 | 0.8200306637806639 |
+--------------------+--------------------+--------------------+


## GBTClassifier

In [12]:

# Train a GBT model.
gbt = GBTClassifier(featuresCol='features', labelCol=label_column, maxIter=100)
# Train model.  This also runs the indexers.
gbt_model = gbt.fit(trainingData)

In [13]:
evaluate_model(gbt_model, trainingData, validation_data=validationData, model_name='GBTClassifier', label_column=label_column)

+--------------------+-----------------+--------------------+
| Model              | GBTClassifier   |                    |
+====================+=================+====================+
| Metric             | Training        | Validation         |
+--------------------+-----------------+--------------------+
| Accuracy           | 1.0             | 0.9949494949494949 |
+--------------------+-----------------+--------------------+
| Weighted Precision | 1.0             | 0.9949792038027332 |
+--------------------+-----------------+--------------------+
| Weighted Recall    | 1.0             | 0.9949494949494949 |
+--------------------+-----------------+--------------------+
| F1 Score           | 1.0             | 0.9949126413961066 |
+--------------------+-----------------+--------------------+


## SVM

In [14]:
lsvc = LinearSVC(featuresCol='features', labelCol=label_column,maxIter=100, regParam=0.1)
# Fit the model
lsvcModel = lsvc.fit(trainingData)



In [15]:
evaluate_model(lsvcModel, trainingData, validation_data=validationData, model_name='SVM', label_column=label_column)

+--------------------+--------------------+--------------------+
| Model              | SVM                |                    |
+====================+====================+====================+
| Metric             | Training           | Validation         |
+--------------------+--------------------+--------------------+
| Accuracy           | 0.9225589225589226 | 0.8787878787878788 |
+--------------------+--------------------+--------------------+
| Weighted Precision | 0.92625602429524   | 0.8762947349334784 |
+--------------------+--------------------+--------------------+
| Weighted Recall    | 0.9225589225589226 | 0.8787878787878788 |
+--------------------+--------------------+--------------------+
| F1 Score           | 0.9086511372225657 | 0.8454545454545455 |
+--------------------+--------------------+--------------------+


### Train the highest performing model on Train + Validation Data and test on Test Data

In [16]:
combined_data = trainingData.union(validationData)
# Train a GBT model.
gbt_model = gbt.fit(combined_data)

In [17]:
evaluate_model(gbt_model, combined_data, test_data=testData, model_name='GBTClassifier', label_column=label_column)

+--------------------+-----------------+--------------------+
| Model              | GBTClassifier   |                    |
+====================+=================+====================+
| Metric             | Training        | Test               |
+--------------------+-----------------+--------------------+
| Accuracy           | 1.0             | 0.9855769230769231 |
+--------------------+-----------------+--------------------+
| Weighted Precision | 1.0             | 0.9876373626373626 |
+--------------------+-----------------+--------------------+
| Weighted Recall    | 1.0             | 0.9855769230769231 |
+--------------------+-----------------+--------------------+
| F1 Score           | 1.0             | 0.9860742705570291 |
+--------------------+-----------------+--------------------+


 ## Save

In [18]:
# model_path = "./gbt_model"
# gbt_model.save(model_path)

